# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [425]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [340]:
df = pd.read_csv('previsao_de_renda.csv')
df = df.drop(columns=['data_ref', 'id_cliente', 'Unnamed: 0'])
df = pd.get_dummies(df, columns=['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], drop_first=True)
df = df.astype(float).dropna()
df.isnull().sum()

posse_de_veiculo                 0
posse_de_imovel                  0
qtd_filhos                       0
idade                            0
tempo_emprego                    0
qt_pessoas_residencia            0
renda                            0
sexo_M                           0
tipo_renda_Bolsista              0
tipo_renda_Empresário            0
tipo_renda_Pensionista           0
tipo_renda_Servidor público      0
educacao_Pós graduação           0
educacao_Secundário              0
educacao_Superior completo       0
educacao_Superior incompleto     0
estado_civil_Separado            0
estado_civil_Solteiro            0
estado_civil_União               0
estado_civil_Viúvo               0
tipo_residencia_Casa             0
tipo_residencia_Com os pais      0
tipo_residencia_Comunitário      0
tipo_residencia_Estúdio          0
tipo_residencia_Governamental    0
dtype: int64

In [342]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               12427 non-null  float64
 1   posse_de_imovel                12427 non-null  float64
 2   qtd_filhos                     12427 non-null  float64
 3   idade                          12427 non-null  float64
 4   tempo_emprego                  12427 non-null  float64
 5   qt_pessoas_residencia          12427 non-null  float64
 6   renda                          12427 non-null  float64
 7   sexo_M                         12427 non-null  float64
 8   tipo_renda_Bolsista            12427 non-null  float64
 9   tipo_renda_Empresário          12427 non-null  float64
 10  tipo_renda_Pensionista         12427 non-null  float64
 11  tipo_renda_Servidor público    12427 non-null  float64
 12  educacao_Pós graduação         12427 non-null  floa

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [344]:
df.head()

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0.0,1.0,0.0,26.0,6.602740,1.0,8060.34,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,28.0,7.183562,2.0,1852.15,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,35.0,0.838356,2.0,2253.89,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,30.0,4.846575,3.0,6600.77,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,33.0,4.293151,1.0,6475.97,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [401]:
# 1)

y = df.loc[:, 'renda']
X = df.drop(columns=['renda'])
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)
train_concat = pd.concat([X_train, y_train], axis=1)

In [403]:
# 2)

md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598555


In [350]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.001
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-2.67972485e+03  1.37860696e+02  2.96334272e+02 -1.05488076e+02
  3.89130561e+01  5.66069086e+02  8.85809517e+01  6.05347977e+03
 -4.32742678e+02  7.29451638e+02 -1.00813232e+03 -7.77356013e+01
  7.31866870e+02  4.41647266e+02  8.43760209e+02  1.40226320e+02
 -3.45168497e+00 -4.04696388e+02 -3.90839017e+02 -1.04256234e+02
 -4.72114694e+02 -5.04093149e+02 -5.79272194e+02  2.67719839e+02
 -3.24955154e+02]

Erro Quadrático Médio (MSE): 44222076.097645596
R²: 0.2925913867701675


In [352]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.005
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-1.48480536e+03  1.68206435e+02  2.82505287e+02  1.78626536e+02
  3.59172066e+01  5.65387539e+02 -2.05386556e+02  5.92374248e+03
 -1.13293661e+02  7.03965884e+02 -3.23213705e+02 -8.58028803e+01
  1.90047035e+02 -2.93981719e+00  3.81765410e+02 -2.87028761e+02
 -2.79277710e+02 -6.81631737e+02 -3.82787876e+02 -3.60474042e+02
 -4.51065603e+02 -4.69383436e+02 -3.61561785e+02  2.03060611e+02
 -2.78770255e+02]

Erro Quadrático Médio (MSE): 44228115.6504076
R²: 0.2924947736750594


In [354]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.01
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-1168.51558724   203.69348556   267.50992162   186.76242271
    33.09250813   564.45953786  -221.72374151  5774.64842943
   -59.04993218   673.02195785  -176.68576366   -96.79987095
    88.01845084  -126.0292619    243.36039396  -376.03728742
  -279.3226347   -678.36781574  -369.68361268  -339.40089979
  -425.5466662   -426.48882145  -239.71233258   165.32629763
  -232.7595755 ]

Erro Quadrático Médio (MSE): 44244303.37636895
R²: 0.2922358229928621


In [356]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.05
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-543.96974327  400.74578124  190.75067866   59.05119298   21.64723129
  557.53891309 -112.37040808 4838.32945746  -12.97027859  505.45072828
  -40.17926308 -133.20805469   14.58148575 -182.21161994  105.78704414
 -276.37335511 -155.80329767 -468.22373222 -287.58606963 -185.65911948
 -307.64083897 -231.86811705  -60.20417202   88.02140777  -97.29114301]

Erro Quadrático Médio (MSE): 44531722.587819524
R²: 0.28763805545840926


In [358]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.1
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-304.94879564  514.73161222  142.90921775  -14.62068788   17.02970618
  550.87125645  -21.83467093 4054.44579072   -7.09319284  394.59889379
  -20.9137492  -132.27375699    6.69473628 -137.55045576   91.51599057
 -171.49969992 -103.19537802 -320.56064164 -224.85646392 -133.32544006
 -223.0110501  -127.41063041  -28.79819835   63.66066566  -53.69135256]

Erro Quadrático Médio (MSE): 45007111.8089832
R²: 0.2800333824225878


### Conclusão ex 2

Podemos observar que quanto mais proximo de zero é o valor do alpha maior é o valor do R², Assim tornando o modelo mais explicativo.

In [360]:
# 3)Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [362]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0.001
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [365]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0.005
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [366]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0.01
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5496.55541775   132.64833415   300.51607458 -1197.93260927
    40.42170719   566.30685747  1188.77788967  6097.15747822
 -1450.13982469   736.80428042 -2180.0060823    -77.49085329
  1596.17739442   996.20173108  1410.75273102   704.74376365
  1100.09857313   693.46495813  -382.47490769  1012.34587471
  -497.22999574  -528.30970719  -667.65197939     0.
  -360.8920597 ]

Erro Quadrático Médio (MSE): 44219178.141448416
R²: 0.2926377445931272


In [369]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0.05
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5496.55541775   132.64833415   300.51607458 -1197.93260927
    40.42170719   566.30685747  1188.77788967  6097.15747822
 -1450.13982469   736.80428042 -2180.0060823    -77.49085329
  1596.17739442   996.20173108  1410.75273102   704.74376365
  1100.09857313   693.46495813  -382.47490769  1012.34587471
  -497.22999574  -528.30970719  -667.65197939     0.
  -360.8920597 ]

Erro Quadrático Médio (MSE): 44219178.141448416
R²: 0.2926377445931272


In [371]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0.1
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5496.55541775   132.64833415   300.51607458 -1197.93260927
    40.42170719   566.30685747  1188.77788967  6097.15747822
 -1450.13982469   736.80428042 -2180.0060823    -77.49085329
  1596.17739442   996.20173108  1410.75273102   704.74376365
  1100.09857313   693.46495813  -382.47490769  1012.34587471
  -497.22999574  -528.30970719  -667.65197939     0.
  -360.8920597 ]

Erro Quadrático Médio (MSE): 44219178.141448416
R²: 0.2926377445931272


### Conclusão ex 3

Diferente do que vimos no exercicio 2, nesse exercicio concluimos que quanto maior o alpha maior é o R² por mais que o aumento seja quase imperceptivel.

In [380]:
# 4)Rode um modelo stepwise. Avalie o R² na fase de testes. Qual o melhor resultado?

md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=.5,
    alpha=0.001
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [384]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=.5,
    alpha=0.005
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [386]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=.5,
    alpha=0.01
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5588.43143966   131.43688901   300.10318157 -1197.94420868
    40.29456529   566.21897871  1188.16524179  6096.61432148
 -1449.13926036   735.65946184 -2181.97628093   -79.8440423
  1595.38142747   995.91726409  1409.22093014   705.8989366
  1099.31815755   694.4664652   -383.9908561   1013.2650445
  -395.76036984  -428.04150888  -566.11600531   370.69356759
  -259.59521485]

Erro Quadrático Médio (MSE): 44222315.876911215
R²: 0.29258755108598566


In [388]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=.5,
    alpha=0.05
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5166.02092361   136.5076196    303.89840065 -1195.28834306
    40.66131016   566.0616004   1188.2236457   6093.23069606
 -1450.90328914   741.0355923  -2182.28142441   -76.98537746
     0.           652.58740466  1066.65630551   362.02961355
  1096.23865325   699.22585036  -386.53819415   998.67281643
  -494.7749466   -529.91500359  -703.0461575      0.
  -364.62741158]

Erro Quadrático Médio (MSE): 44211074.17527468
R²: 0.2927673815522778


In [390]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=.5,
    alpha=0.1
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[-5258.45846433   135.28902039   303.48203416 -1195.30131677
    40.53350028   565.97346571  1187.60863491  6092.68716495
 -1449.89803008   739.88372049 -2184.25897494   -79.34896174
     0.           652.47380055  1065.29023302   363.36140332
  1095.45680725   700.22878869  -388.05864124   999.60279553
  -392.86922181  -429.21385098  -601.05484293   372.29123619
  -262.89211225]

Erro Quadrático Médio (MSE): 44214205.8480692
R²: 0.2927172850280041


### Ex 5

Com base nos meus estudos, cheguei a conclusão de que usando o stepwise, conseguimos modelos de melhor qualidade.  
Se encaichando no melhor formato para encontrar o melhor modelo. 

In [407]:
# 6)

y = df.loc[:, 'renda']
X = df.drop(columns=['renda'])
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, np.log(y), random_state=100)
train_concat = pd.concat([X_train, y_train], axis=1)

In [417]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0,
    alpha=0.001
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[ 4.41636484e+00  4.12732718e-02  8.25047640e-02 -1.02340239e+00
  7.72241942e-03  6.14192597e-02  1.05333835e+00  7.99324180e-01
  4.47804520e-02  1.60452242e-01 -1.28361092e-01  5.09653518e-02
  3.71412257e-01  3.33813212e-01  4.36384907e-01  3.64343497e-01
  1.08190763e+00  1.01785686e+00 -3.28429404e-03  1.08737696e+00
  1.53256754e-01  1.90689755e-01  1.39076241e-01  2.19146499e-01
  1.24526908e-01]

Erro Quadrático Médio (MSE): 0.5143281931801805
R²: 0.3652273696196071


In [421]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=1,
    alpha=0
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[ 6.65334110e+00  3.31673207e-02  7.54119425e-02 -2.58806753e-01
  5.23075406e-03  6.16839294e-02  2.75926325e-01  7.88071087e-01
  1.64801796e-01  1.50948683e-01 -2.91152246e-01  4.76156194e-02
  2.17469218e-01  8.53794604e-03  1.08092209e-01  2.93884587e-02
  3.25903857e-01  2.40786089e-01 -1.44829316e-02  3.54615950e-01
 -8.49487796e-02 -6.09209463e-02 -1.04288999e-01  3.89720047e-02
 -1.07790082e-01]

Erro Quadrático Médio (MSE): 0.5137703871224115
R²: 0.3659158016815224


In [423]:
md = sm.OLS(y_train, X_train)
reg = md.fit_regularized(
    method='elastic_net',
    refit=True,
    L1_wt=0.5,
    alpha=0
)

print("Coeficientes do Modelo Regularizado:")
print(reg.params)

y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nErro Quadrático Médio (MSE):", mse)
print("R²:", r2)

Coeficientes do Modelo Regularizado:
[ 6.65334110e+00  3.31673207e-02  7.54119425e-02 -2.58806753e-01
  5.23075406e-03  6.16839294e-02  2.75926325e-01  7.88071087e-01
  1.64801796e-01  1.50948683e-01 -2.91152246e-01  4.76156194e-02
  2.17469218e-01  8.53794604e-03  1.08092209e-01  2.93884587e-02
  3.25903857e-01  2.40786089e-01 -1.44829316e-02  3.54615950e-01
 -8.49487796e-02 -6.09209463e-02 -1.04288999e-01  3.89720047e-02
 -1.07790082e-01]

Erro Quadrático Médio (MSE): 0.5137703871224115
R²: 0.3659158016815224


In [427]:
# 7)

clf = DecisionTreeRegressor(max_depth=4)

clf.fit(X_train, y_train)

predict = clf.predict(X_train)
R2 = clf.score(X_train, y_train)
mse = mean_squared_error(y_train, predict)
print(f'MSE: {mse}')
print(f'R²: {R2}')

MSE: 0.5096125992401578
R²: 0.36972784503895073
